# Test formation of quasicrystal by projection of high-dimentional unit cells

In [1]:
%matplotlib inline
from neurotools.nlab import *
from edges_and_triangles import *

Loading nlab namespace
Could not locate the spectrum module, please install it
Multitaper methods will not work
Could not locate the spectrum module, please install it
Multitaper methods will not work


In [2]:
# Generate a collection of paths in M dimensions
# Use the unit vector projection to identify the point location
# TODO: we need a better datastructure for detecting identical points
# I don't know how to do this theoretically unless they occupy
# The same location in the higher-dimensional lattice?

# Hypothesis: two projections are identical if and only if their unit vectors agree? 
# This may speed things up?

# Still not fast enough. Let's grow the graph from a seed
# Using the high-dimensional representation
# Iteratively, but only adding points that are close to the plane?
# This *may* involve computing distance to a hyperplane. Yuck.
# Hmm.

# What if we test points by projecting into 2D and out again and seeing whether this operation is a fixed point?

In [3]:
# Dimensionality
M = 7
# Scale 
K = 10
# Maximum radius of pattern
R = 3
# Number of nearest-neighbor edge hops to compute when defining the lattice tiling
I = 10
# number of hops to expand the graph before culling back to 2d
H = 3
# Unit vectors; make explicit no using complex this time
phases  = arange(M)*pi/M
units2d = np.float64([cos(phases),sin(phases)])

In [ ]:
# To check whether a point is on the hyperplane, project from M -> 2 -> M?
# This is linear, combine
project2d = units2d.T.dot(units2d)
project2d.shape
# Define the M-D basis
basis = np.int32(np.eye(M))

In [ ]:
# Don't use a grid, use a seed.
points = np.zeros((1,M),dtype=np.int32)
# Define units step vectors in all M dimensions
steps = np.concatenate([basis,-basis]).T

# Store located points in a set
hyperpoints = set(map(tuple,points))

for iteration in range(I):
    
    # Track newly created points
    newset = set()
    newhyperpoints = set(hyperpoints)
    
    changed = False
    for hop in range(H):
        # Use newfound points as seed for next iteration
        points = np.int32(array(list(newhyperpoints)))
        
        # Remove points outside radius
        points2d = units2d.dot(points.T)
        radius = sum(points2d**2,axis=0)**0.5
        points = points[radius<2*R,:]
        
        # Generate adjacent vectors
        adjacent = (points[...,None] + steps[None,:]).transpose((0,2,1))
        npt,nadj,ndim = adjacent.shape
        adjacent = adjacent.reshape((npt*nadj,ndim))
        # Convert to set
        adjacentset = set(map(tuple,adjacent))
        # Add to hyperpoint set
        newset |= (adjacentset-hyperpoints)
        newhyperpoints |= adjacentset
    
    hyperpoints = newhyperpoints
    
    if len(newset)<=0:
        print("No new points found, iteration %d"%iteration)
        break
        
    # Remove existing vectors
    new = array(list(newset))
    # Form projection onto 2D slice
    new2d = new.dot(project2d)
    # Remove points outside radius
    radius = sum(new2d**2,axis=1)**0.5
    new2d = new2d[radius<R]
    # Quantize projection in hyperspace
    new = np.int32(np.round(units2d.dot(new2d.T)))
    # Add new points
    newset = set(map(tuple,adjacent)) - hyperpoints
    hyperpoints |= newset

    
points = units2d.dot(array(list(hyperpoints)).T)
radius = sum(points**2,axis=0)**0.5
figure(figsize=(15,15))
scatter(*points[:,radius<R],s=1)
force_aspect()

# Complete tiling using Voronoi regions

In [ ]:
from scipy.spatial import Voronoi, voronoi_plot_2d
figure(figsize=(55,55))
vor = Voronoi(points.T,qhull_options="Qc")

v = vor.vertices
v = v[:,0]+1j*v[:,1]
v = array(v)
r = array(vor.regions)

ee = concatenate([array(list(zip(array(e)[arange(-1,len(e))],e))) for e in r])

v[abs(v)>R]=NaN

#ok = (abs(diff(v[ee],1,1))<1).squeeze()
#print(ok.shape,ee.shape)
plot_edges(v,ee,color=BLACK,lw=0.5)

xlim(-R,R)
ylim(-R,R)
force_aspect()

# Complete tiling using Delaunay triangulation

In [ ]:
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay

pp = points[:,radius<R].T
tri = Delaunay(pp,furthest_site=False, incremental=False, qhull_options=None)

figure(figsize=(15,15))
force_aspect()
plt.triplot(pp[:,0], pp[:,1], tri.simplices.copy())
plt.plot(pp[:,0], pp[:,1], 'o', markersize = 0, linewidth=0.5)
force_aspect()

In [ ]:
pp.shape

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.tri as tri
import numpy as np
import math
pp = points[:,radius<R]
# Create the Triangulation; no triangles so Delaunay triangulation created.
triang = tri.Triangulation(pp[0],pp[1])
# Plot the triangulation.
plt.figure(figsize=(55,55))
plt.gca().set_aspect('equal')
plt.triplot(triang, 'bo-', lw=0.5,markersize=0,color=BLACK)
plt.title('triplot of Delaunay triangulation')

xlim(-R,R)
ylim(-R,R)

# Complete tiling using adjacency graph of hypercrystal

Edges exist if two points are separated by edit distance of 1. Neighborhood-expand existing point set to find them

In [ ]:
# Convert point set to array
hyperlist = list(hyperpoints)
x = np.int8(hyperlist)
# Create ID lookup table
hyperid = dict(zip(hyperpoints,range(len(hyperlist))))
# Generate adjacent vectors
adjacent = (x[...,None] + steps[None,:]).transpose((0,2,1))
npt,nadj,ndim = adjacent.shape
# Find all neighbors in the hyperpoints set
neighbors = [set(map(tuple,adj))&hyperpoints for adj in adjacent]
# Convert to indecies into point array
neighbor_id = [[hyperid[p] for p in nn] for nn in neighbors]
# Convert to edge-indicies
ee = array([(i,a) for (i,nn) in enumerate(neighbor_id) for a in nn])
# Convert points to complex for faster distance computation
v = array(points.T)
v = v[:,0]+1j*v[:,1]
v = array(v)
# Remove edges that are "long" in the 2D projection
length = abs(diff(v[ee],1,1))[:,0]
ee = ee[length<1,:]

# Render 
figure(figsize=(14,14))
v[abs(v)>R]=NaN
plot_edges(v,ee,color=BLACK,lw=0.5)
#xlim(-R,R)
#ylim(-R,R)